In [ ]:
!pip install torch torchvision torchaudio
!pip install diffusers
!pip install gradio
!pip install pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 127.9/127.9 MB 149.8 MB/s eta 0:00:01

In [ ]:

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
import gradio as gr
from PIL import Image

In [ ]:

# Load Stable Diffusion model
authorization_token = ""
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16, use_auth_token=authorization_token
)
pipe.to(device)

In [ ]:
# Function to generate the image
def generate_image(prompt):
    with autocast("cuda"):
        image = pipe(prompt, guidance_scale=8.5).images[0]
    return image


In [ ]:

# Create the Userinterface
def gradio_interface():
    with gr.Blocks(css="""
        .gradio-container {
            background: linear-gradient(to right, #4B0082, #8A2BE2); /* Gradient from Indigo to BlueViolet */
            color: white;
            font-family: 'Arial', sans-serif;
            height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            text-align: center;
        }
        .gr-button {
            padding: 14px 35px;
            font-size: 18px;
            font-weight: bold;
            border-radius: 12px;
            color: white;
            border: none;
            cursor: pointer;
            margin: 10px;
            transition: background-color 0.3s ease;
        }
        .button-start {
            background-color: #FF6347; /* Tomato Red */
        }
        .button-start:hover {
            background-color: #FF4500; /* OrangeRed */
        }
        .button-generate {
            background-color: #00BFFF; /* Deep Sky Blue */
        }
        .button-generate:hover {
            background-color: #1E90FF; /* Dodger Blue */
        }
        .button-return {
            background-color: #9932CC; /* Dark Orchid */
        }
        .button-return:hover {
            background-color: #8A2BE2; /* BlueViolet */
        }
        .gr-label {
            font-size: 24px;
            font-weight: bold;
            color: white;
            text-align: center;
            margin-bottom: 20px;
        }
        .gr-textbox {
            background-color: rgba(255, 255, 255, 0.85);
            border-radius: 12px;
            padding: 12px;
            font-size: 18px;
            width: 100%;
            margin: 10px 0;
        }
        .gr-textbox input {
            background-color: rgba(255, 255, 255, 0.85);
            color: #333;
        }
        .gr-row {
            justify-content: center;
            align-items: center;
            margin-top: 20px;
        }
        .gr-markdown {
            text-align: center;
            margin-bottom: 30px;
        }
    """) as demo:

        gr.Markdown(
            """
            # Text-to-Image Using Stable Diffusion
            **Welcome to the image generation app!**
            Click **Start** to begin.
            """
        )

        # Layout elements
        with gr.Row():
            welcome_message = gr.Label(
                "Welcome to the generation app! Click 'Start' to enter your text prompt.",
                label=None,
            )
            output_image = gr.Image(label="Generated Image", visible=False)

        with gr.Row():
            start_button = gr.Button("Start", elem_classes=["gr-button", "button-start"])
            prompt_input = gr.Textbox(
                placeholder="Enter your prompt here...", visible=False, label="Enter your text prompt"
            )
            submit_button = gr.Button("Generate Image", visible=False, elem_classes=["gr-button", "button-generate"])
            next_button = gr.Button("Return & Enter New Prompt", visible=False, elem_classes=["gr-button", "button-return"])

        # Button interactions
        def start():
            return {
                welcome_message: "Please enter your text prompt below and click 'Generate Image'.",
                prompt_input: gr.update(visible=True),
                submit_button: gr.update(visible=True),
            }

        def submit(prompt):
            image = generate_image(prompt)
            return {
                output_image: gr.update(value=image, visible=True),
                next_button: gr.update(visible=True),
            }

        def return_to_start():
            return {
                prompt_input: gr.update(value="", visible=True),
                output_image: gr.update(visible=False),
                submit_button: gr.update(visible=True),
                next_button: gr.update(visible=False),
            }

        # Connect buttons to functions
        start_button.click(start, outputs=[welcome_message, prompt_input, submit_button])
        submit_button.click(submit, inputs=[prompt_input], outputs=[output_image, next_button])
        next_button.click(return_to_start, outputs=[prompt_input, output_image, submit_button, next_button])

    return demo

# Launch the Gradio app
demo = gradio_interface()
demo.launch(debug=True)
